# Setup

In [1]:
import os

from Db import DataBase
from Category import Category
from Feature import Feature

import pandas as pd
import numpy as np

from Trainset import Trainset
from Testset import Testset
from Trainer import Trainer

# Load Data

In [2]:
# Load Data
file_path = os.getcwd() + "/../data/"
dbase = DataBase(file_path)

# Data Preprocessing

In [3]:
baskets_data, coupons_data = dbase.load_basket_coupon_data()

In [4]:
# orignal_price
original_price = dbase.original_price(baskets_data)

# Product Category Table
catClass = Category(baskets_data)
prods_cat_table = catClass.generate_product_category_table()

baskets_train, baskets_test, coupons_train, coupons_test = dbase.split_data(baskets_data, coupons_data)

/Users/jungwoojamesjeong/Project/hu/mlim_group_3_final_assignment/project/Db.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baskets_train[column] = baskets_train[column].cat.remove_unused_categories()
/Users/jungwoojamesjeong/Project/hu/mlim_group_3_final_assignment/project/Db.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baskets_test[column] = baskets_test[column].cat.remove_unused_categories()
/Users/jungwoojamesjeong/Project/hu/mlim_group_3_final_assignment/project/Db.py:38: SettingWithCopyW

In [5]:
baskets_train, baskets_test, coupons_train, coupons_test = dbase.generate_split_data_with_category(prods_cat_table, baskets_train, baskets_test, coupons_train, coupons_test)

In [6]:
print(prods_cat_table.shape)

(250, 2)


In [7]:
baskets_test.head(2)

,week,shopper,product,price,target,category
0,89,0,67,637,1,12
1,89,0,71,629,1,24


In [8]:
coupons_train.head(2)

,week,shopper,product,discount,category
0,0,0,35,35,8
1,0,0,193,40,0


In [9]:
coupons_test.head(2)

,week,shopper,product,discount,category
0,89,0,131,30,7
1,89,0,16,25,18


# Feature Engineering

In [10]:
# Define feat
print("feature engineering begin")
feat = Feature(baskets_train, coupons_train)

feature engineering begin


In [11]:
total_count_of_product = feat.total_count_of_product()
reordered_category = (baskets_train.groupby(['shopper']).category.value_counts()>1).astype(int)
reordered_product = feat.reordered_product()
category_count = feat.category_count()
reordered_category = feat.reordered_category()
coupon_in_same_category = feat.coupon_in_same_category()
average_price_per_shopper = feat.average_price_per_shopper()
average_basket_size = feat.average_basket_size()
unique_products_per_shopper = feat.unique_products_per_shopper()
unique_categories_per_shopper = feat.unique_categories_per_shopper()
ratio_of_reordered_products_per_shopper = feat.ratio_of_reordered_products_per_shopper()
ratio_of_reordered_categories_per_shopper = feat.ratio_of_reordered_categories_per_shopper()

In [12]:
reordered_product = reordered_product.to_frame('reordered_product').reset_index()
reordered_category = reordered_category.to_frame('reordered_category').reset_index()
unique_products_per_shopper = unique_products_per_shopper.to_frame('unique_products_per_shopper').reset_index()
unique_categories_per_shopper = unique_categories_per_shopper.to_frame('unique_categories_per_shopper').reset_index()

In [13]:
print("feature engineering finished")

feature engineering finished


# Training Set

In [14]:
# Train Table
train_t = Trainset(baskets_train, coupons_train, original_price)

In [15]:
full_df_train = train_t.generate_full_df_train()

In [16]:
## featureless_training_set
num_weeks = 88
num_shoppers = 100

df1 = pd.DataFrame({'key': np.ones(num_weeks), 'week': list(range(num_weeks))})
df2 = pd.DataFrame({'key': np.ones(num_shoppers), 'shopper': list(range(num_shoppers))})
df3 = pd.DataFrame({'key': np.ones(250), 'product': list(range(250))})

featureless_training_set = (pd
    .merge(df1, df2, on='key')
    .merge(df3, on='key')
    .merge(prods_cat_table, on='product')
    .merge(full_df_train, on=['week', 'shopper', 'product', 'category'], how='left')[full_df_train.columns]
    )

print("----- featureless training set -----")


----- featureless training set -----


In [17]:
featureless_training_set.head(2)

,week,shopper,product,price,target,category,discount,coupon
0,0,0,0,NaN,NaN,2,NaN,NaN
1,0,1,0,NaN,NaN,2,NaN,NaN


In [18]:
## Generate training set
training_set = train_t.populate_features(
    featureless_training_set,
    original_price,
    total_count_of_product,
    reordered_product,
    category_count,
    reordered_category,
    coupon_in_same_category,
    average_price_per_shopper,
    average_basket_size,
    unique_products_per_shopper,
    unique_categories_per_shopper
#     ratio_of_reordered_products_per_shopper,
#     ratio_of_reordered_categories_per_shopper,
)

In [19]:
print("===== training_set =====")
training_set.head(2)

===== training_set =====


,week,shopper,product,price,target,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,0,688.0,0.0,2,0.0,No,688,0,0.0,8,1.0,Yes,587.203947,8.539326,54,25,1,1
1,0,1,0,688.0,0.0,2,0.0,No,688,6,1.0,28,1.0,Yes,584.065350,7.393258,71,24,1,1


In [20]:
# X_train, y_train
X_train, y_train = train_t.split_trainingset_to_X_train_and_y_train(training_set)

In [21]:
X_train.head(2)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,2,0.0,No,688,0,0.0,8,1.0,Yes,587.203947,8.539326,54,25,1,1
1,1,0,688.0,2,0.0,No,688,6,1.0,28,1.0,Yes,584.065350,7.393258,71,24,1,1


In [22]:
y_train.head(2)

0    0.0
1    0.0
Name: target, dtype: float64

# Testing Set

In [23]:
# testing table
test_t = Testset(baskets_test, coupons_test)

In [24]:
# full_df_test
full_df_test = test_t.generate_full_df_test()

In [25]:
# Generate testing set
testing_set = test_t.generate_training_set(
    prods_cat_table,
    original_price,
    total_count_of_product,
    reordered_product,
    category_count,
    reordered_category,
    coupon_in_same_category,
    average_price_per_shopper,
    average_basket_size,
    unique_products_per_shopper,
    unique_categories_per_shopper,
    training_set
#     ratio_of_reordered_products_per_shopper,
#     ratio_of_reordered_categories_per_shopper,
)

In [26]:
X_test, y_test = test_t.split_testingset_to_X_test_and_y_test(testing_set)

In [27]:
print(X_test.shape)
X_test.head(2)

(25000, 18)


,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,2,0.0,No,688,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458
1,0,1,560.0,2,0.0,No,560,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458


In [28]:
y_test.head(2)

0    0.0
1    0.0
Name: target, dtype: float64

In [29]:
# Duplicate all for X_test entries (each product for each shoppers for each week)
# => updated values: 
# 1) discount 30, 
# 2) coupon: Yes
# 3) price: original_price * (1 - (discount / 100))
# 4) coupon in same category (?)


# 25000 entries

In [30]:
X_test_w_coupons = X_test.copy()

In [31]:
X_test_w_coupons.head(2)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,2,0.0,No,688,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458
1,0,1,560.0,2,0.0,No,560,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458


In [32]:
# 1) discount 30%
X_test_w_coupons.loc[:,"discount"] = 30
X_test_w_coupons.head(2)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,2,30,No,688,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458
1,0,1,560.0,2,30,No,560,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458


In [33]:
# 2) coupons = Yes
X_test_w_coupons.loc[:, "coupon"] = "Yes"
X_test_w_coupons.head(2)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,2,30,Yes,688,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458
1,0,1,560.0,2,30,Yes,560,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458


In [34]:
# 3) price = original_price * (1 - (discount / 100))
X_test_w_coupons.loc[:, "price"] = X_test_w_coupons["original_price"] * (1 - (X_test_w_coupons["discount"] / 100))
X_test_w_coupons.head(2)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,481.6,2,30,Yes,688,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458
1,0,1,392.0,2,30,Yes,560,0,0.0,8,1.0,No,587.203947,8.539326,54,25,89,458


In [35]:
# Combine X_test AND X_test_w_coupons
new_X_test = pd.concat([X_test, X_test_w_coupons])

In [36]:
print(new_X_test.shape)
new_X_test.sample(10)

(50000, 18)


,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
12738,73,128,441.0,15,30.0,Yes,630,7,1.0,22,1.0,No,570.878289,6.831461,79,24,4,81
2400,40,20,563.0,11,0.0,No,563,0,0.0,3,1.0,No,601.604651,6.764045,57,24,89,100
17091,9,171,639.0,3,0.0,No,639,0,0.0,5,1.0,No,570.781208,8.370787,83,25,89,638
22528,52,228,531.0,4,0.0,No,531,3,1.0,19,1.0,No,573.565619,6.078652,74,25,9,97
7906,90,76,407.4,24,30.0,Yes,582,32,1.0,56,1.0,No,578.817803,8.078652,69,25,3,265
5556,55,56,532.7,22,30.0,Yes,761,0,0.0,16,1.0,No,566.839150,7.404494,59,23,89,47
11432,43,112,727.0,1,0.0,No,727,1,0.0,17,1.0,No,564.316456,7.101124,95,25,69,229
13974,97,134,528.0,7,0.0,No,528,1,0.0,53,1.0,No,592.612587,8.033708,79,25,59,91
10019,1,109,466.9,10,30.0,Yes,667,0,0.0,0,0.0,No,584.065350,7.393258,71,24,89,881
17553,55,173,535.5,3,30.0,Yes,765,0,0.0,30,1.0,No,566.839150,7.404494,59,23,89,320


# Trainer

In [37]:
trainer = Trainer(X_train, y_train, new_X_test, y_test)

In [38]:
model = trainer.fit_model()

/Users/jungwoojamesjeong/opt/anaconda3/envs/hu-bads/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [39]:
predictions = trainer.predict()

/Users/jungwoojamesjeong/opt/anaconda3/envs/hu-bads/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ValueError: train and valid dataset categorical_feature do not match.

In [ ]:
predict_prob = trainer.predict_proba()